In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 77.8MB/s 
     |████████████████████████████████| 6.8MB 49.1MB/s 
     |████████████████████████████████| 983kB 47.5MB/s 
     |████████████████████████████████| 266kB 31.3MB/s 
     |████████████████████████████████| 1.9MB 16.2MB/s 
     |████████████████████████████████| 1.2MB 39.6MB/s 
     |████████████████████████████████| 471kB 33.7MB/s 
     |████████████████████████████████| 3.2MB 45.3MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.0-cp37-none-any.whl size=25276917 sha256=4243e7f4faaf990444c5e0430d6cebfcd5fed787d779d1a9d3b8ca9c8d896b41
  Stored in directory: /root/.cache/pip/wheels/fc/e5/c7/c17fc12743a31df1deacb71ce327f946528774d494d3038995
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=effedb9bf1b74e8632fa5b6dcf5d2c07f35a83f1cced755d647621cf2b55f30f
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import  text

In [ ]:
data_train =pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype='str')

In [ ]:
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [ ]:
data_test=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype='str')
data_test

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
...,...,...
24995,This fanciful horror flick has Vincent Price p...,neg
24996,"The Intruder (L'Intrus), a film directed by Fr...",pos
24997,Holy crap. This was the worst film I have seen...,neg
24998,Clocking in at an interminable three hours and...,neg


In [ ]:
(X_train,y_train),(X_test,y_test),preprocess=text.texts_from_df(train_df=data_train,
                   text_column='Reviews',
                   label_columns='Sentiment',
                   val_df=data_test,
                   maxlen=400,
                  preprocess_mode='bert',)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
X_train[0].shape
print(X_test[0].shape)

(25000, 400)


In [ ]:
model=text.text_classifier(name ='bert',
                           train_data=(X_train,y_train),
                           preproc=preprocess,
                           metrics=['accuracy'],
                           verbose=1)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
#learning rate
learner=ktrain.get_learner(model=model,
                           train_data=(X_train,y_train),
                           val_data=(X_test,y_test),
                           batch_size=6)

In [ ]:
#learner.lr_find()
#learner.lr_plot()
#optimal learning 2e-5

simulating training for different learning rates... this may take a few moments...
Epoch 1/1024


In [ ]:
learner.fit_onecycle(lr=2e-5 ,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
   6/4167 [..............................] - ETA: 32:39:50 - loss: 0.8666 - accuracy: 0.4250

KeyboardInterrupt: ignored

In [ ]:
predictor=ktrain.get_predictor(model=learner.model,preproc=preprocess)

In [ ]:
data=['And while the action remains flawlessly slick and polished as ever with Yen in his quick-fisted, fighting element, this was one fight too many',
      'The Finale has its flaws, but it does bring a satisfying conclusion to this epic saga',
      'As a Chinese, this was nothing but embarrassing. It has no plot, no acting skills, and no moral. It portrayed Americans as brainless prejudiced hatred-filled demons. The essence of the movie is pretty much "hey, Chinese pig you want to fight?" "Sure, lets fight." Please Avoid if you cherish your time and your life.',
      'I like how the setting is displayed and theres a lot of action scenes between Donnie yen and Scott Adkins. This bring a lot of excitement to us, hope to make more of movies like this in the future']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'neg', 'neg']